In [28]:
import numpy as np
import pandas as pd

from scipy import stats, special
from scipy.optimize import minimize 

In [94]:
df = pd.read_csv('results/simdata.csv')
df.head()

,Unnamed: 0,sub,choices,values,risk,ambiguity,riskTol,ambTol
0,0,1,0,5,0.25,0.0,0.362151,-0.172864
1,1,1,0,5,0.25,0.0,0.362151,-0.172864
2,2,1,0,5,0.25,0.0,0.362151,-0.172864
3,3,1,0,5,0.50,0.0,0.362151,-0.172864
4,4,1,0,5,0.50,0.0,0.362151,-0.172864


In [95]:
df10 = df[df['sub']==1]
#df10 = df10.sample(frac=1)
df10.head()


,Unnamed: 0,sub,choices,values,risk,ambiguity,riskTol,ambTol
0,0,1,0,5,0.25,0.0,0.362151,-0.172864
1,1,1,0,5,0.25,0.0,0.362151,-0.172864
2,2,1,0,5,0.25,0.0,0.362151,-0.172864
3,3,1,0,5,0.50,0.0,0.362151,-0.172864
4,4,1,0,5,0.50,0.0,0.362151,-0.172864


In [96]:
values = df10['values'].to_numpy()
risk = df10['risk'].to_numpy()
ambiguity = df10['ambiguity'].to_numpy()
choices = df10['choices'].to_numpy()

In [99]:
α = df.riskTol[0]
β = df.ambTol[0]
svLotto = (values ** α) * (risk-(β * (ambiguity/2)))
svRef = 5 ** α
p = special.expit(svLotto - svRef)
size = len(p)
mu = np.random.binomial(1,p,size)
tr = pd.DataFrame({'real': choices, 'estimated': mu})
tr['diff'] = (tr.real - tr.estimated) ** 2
np.sum(tr['diff'])

46

In [104]:
np.sum((choices-p)**2)

19.917749480269688

In [89]:
def MLE_riskamb(parameters):
    # extract parameters
    α, β, γ = parameters
    
    
    svLotto = (df['values'] ** α) * (risk-(β * (ambiguity/2)))
    svRef = 5 ** α
    p = 1/(1+np.exp((svRef - svLotto) * γ))
    size = len(p)
    mu = np.random.binomial(1,p,size)

    
    # Calculate the log-likelihood for normal distribution
    LL = np.sum(stats.norm.logpdf(choices, p))
    # Calculate the negative log-likelihood
    neg_LL = -1*LL
    return neg_LL 

In [116]:
def MLE_riskamb(parameters):
    # extract parameters
    α, β, γ = parameters
    
    
    svLotto = (db_sub['values'].values ** α) * (db_sub['risk'].values-(β * (db_sub['ambiguity'].values/2)))
    svRef = 5 ** α
    p = 1/(1+np.exp((svRef - svLotto) * γ))
    size = len(p)
    mu = np.random.binomial(1,p,size)

    
    # Calculate the log-likelihood for normal distribution
    LL = np.sum(stats.norm.logpdf(db_sub.choices, p))
    # Calculate the negative log-likelihood
    neg_LL = -1*LL
    return neg_LL 

In [117]:
mle_model = minimize(MLE_riskamb, np.array([1,1,1]), method='L-BFGS-B')
mle_model.x

array([0.57279976, 0.02832813, 0.89806227])

In [112]:
db_sub = df